In [10]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from gensim.models import Phrases
from gensim.models.phrases import Phraser
from nltk.corpus import stopwords

#df = pd.read_csv(r'/Users/yuser/Downloads/texts.csv')
#df = df.loc[2141:2142]['EN']
#print(list(df))

df = [["English texts for beginners to practice reading and comprehension online and for free. Practicing your comprehension of written English will both improve your vocabulary and understanding of grammar and word order+A8. The texts below are designed to help you develop while giving you an instant evaluation of your progress. Prepared by experienced English teachers, the texts, articles and conversations are brief and appropriate to your level of proficiency. Take the multiple-choice quiz following each text, and you'll get the results immediately. You will feel both challenged and accomplished! You can even download (as PDF) and print the texts and exercises. It's enjoyable, fun and free. Good luck!"],
      ["These English bob short stories for beginners have been collected from diverse sources. We formed a team of experts who are entrusted with the responsibility of collecting these short stories from all sources possible. The team did a wonderful job of collecting these short stories by going through various materials. AS usual, our work of hard-time has produced desired results. We have given the large collection of those short stories. As per the requirements of our regular readers, we have added these stories of high quality. You are one of those valuable readers. All people of all backgrounds with assorted abilities will like these stories of amusing nature. You are welcome to share with us any such short story (short stories) you have with you. Your story will be presented here with acknowledgement."],
      ["Mr. Smith leads them to the back of store where the two dogs are. One of them is a very big bulldog named Buster. The other is a very tiny chihuahua named Teacup. April wants Teacup. Cody wants Buster. They walk outside to discuss. They canХt agree on a dog. April suggests they race home for it. The winner of the race chooses the dog. Cody agrees, then tells April her shoelace is untied. When April looks down, he runs off and gets a head start."]]

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df))

# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=1, threshold=2) # higher threshold fewer phrases. 5,100
trigram = Phrases(bigram[data_words], threshold=2)  #100

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words][0]])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
#print(data_words_bigrams)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized)


[nltk_data] Downloading package stopwords to /Users/yuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/Users/yuser/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/yuser/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/Users/yuser/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:81: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/Users/yuser/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])

[['english', 'text', 'beginner', 'practice', 'reading', 'comprehension', 'online', 'free', 'practicing', 'comprehension', 'write', 'english', 'improve', 'vocabulary', 'understanding', 'grammar', 'word', 'order', 'text', 'design', 'help', 'develop', 'give', 'instant', 'evaluation', 'progress', 'prepare', 'experience', 'english', 'teacher', 'text', 'article', 'conversation', 'brief', 'appropriate', 'level', 'proficiency', 'take', 'multiple', 'choice', 'quiz', 'follow', 'text', 'get', 'result', 'immediately', 'feel', 'challenge', 'accomplish', 'even', 'download', 'pdf', 'print', 'text', 'exercise', 'enjoyable', 'fun', 'free', 'good', 'luck'], ['english', 'bob', 'short_stori', 'beginner', 'collect', 'diverse', 'source', 'form', 'team', 'expert', 'entrust', 'responsibility', 'collecting', 'short_stori', 'source', 'possible', 'team', 'wonderful', 'job', 'collecting', 'short_storie', 'go', 'various', 'material', 'usual', 'work', 'hard', 'time', 'produce', 'desire', 'result', 'give', 'large', 